In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal
import sklearn as sk
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts
import bk.load
import bk.compute
import bk.plot

In [5]:
%matplotlib qt
%run set_mpl.ipynb

Exception: File `'set_mpl.ipynb.py'` not found.

In [100]:
g = sp.signal.gaussian(100,10)
g = g/np.sum(g)

In [101]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
runintervals = bk.load.run_intervals()
laps = bk.load.laps()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [102]:
t, binned = bk.compute.binSpikes(neurons)
z_binned = sp.stats.zscore(binned,1)
        
    
t = nts.Ts(t,time_units='s')
t_safe = laps['safe'].in_interval(t)>=0
t_danger = laps['danger'].in_interval(t)>=0

z_safe = z_binned[:,t_safe]
z_danger = z_binned[:,t_danger]
z_fused = np.hstack((z_safe,z_danger)).T

In [103]:
z_sm = np.apply_along_axis(lambda m: np.convolve(m, g, mode='same'), axis=0, arr=z_fused)
y = np.hstack((np.ones(np.shape(z_safe)[1]),2*np.ones(np.shape(z_danger)[1])))

In [104]:
clf = LinearDiscriminantAnalysis()
clf.fit(z_sm,y)
z_sm = np.apply_along_axis(lambda m: np.convolve(m, g, mode='same'), axis=1, arr=z_binned)

plt.plot(clf.predict(z_sm.T))

In [105]:
all_laps = laps['safe'].union(laps['danger'])

In [106]:
t = nts.Ts(t,time_units='s')
t_run = all_laps.in_interval(t)>=0

In [107]:
plt.imshow(normal_run)

In [108]:
normal_run = z_binned[:,t_run]
normal_run = np.apply_along_axis(lambda m: np.convolve(m, g, mode='same'), axis=1, arr=normal_run) 

In [113]:
plt.plot(t[t_run].as_units('s').index,clf.predict(normal_run.T))
bk.plot.intervals(laps['danger'],col = 'red')
bk.plot.intervals(laps['safe'],col = 'green')


In [110]:
plt.plot()

[]

In [111]:
normal_run.shape

(252, 16501)

In [112]:
t[t_run]

TypeError: __init__() got multiple values for keyword argument 'index'